In [ ]:
from numpy import mean
from numpy import std
from numpy import dstack
import numpy as np
import pandas as pd
import re
import json
from pandas import json_normalize
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from keras.layers import ConvLSTM2D
from keras.optimizers import Adam
from matplotlib import pyplot

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [ ]:
def loadData(path):
    with open(path,'r') as handle:
        text_data = handle.read()
        json_data = json.loads(text_data)
        df = pd.DataFrame(json_data)
    return df


#Adding frame numbers for time series plots
def LabelFrame(data):   
    df = pd.DataFrame(data)
    for i in range(len(df.ObjectList.index)):
        for j in df['ObjectList'][i]:
            j['Frame'] = i
            
    df.to_json('temp.json',orient = 'records')
    with open('temp.json') as file:
        data2 = file.read()
        jasondata = json.loads(data2)
    return jasondata


def openfile(path):
    with open(path, 'r') as handle:
        text_data = handle.read()
        text_data = '[' + re.sub(r'\}\s\{','},{', text_data) + ']'
        json_data = json.loads(text_data)
        json_data = LabelFrame(json_data)
    return json_data


#Seperating pk array values into columns
def mergedframe1(df_1):
        frames = [df_1]
        df_merged = pd.concat(frames,ignore_index = True)
        df_classify = pd.DataFrame()
        df_classify[['ObjType','pkArray','dpplIdx']] = df_merged[['ObjType','pkArray','dpplIdx']]
        
        df_classify['pkArray'] = df_classify['pkArray'].apply(lambda pkarray:np.ravel(np.transpose(pkarray)))
        
        all_list = []

        for i in range(35):
            for j in range(len(df_classify.index)):
                all_list.append(df_classify.pkArray[j][i])
        chunk = [all_list[i:i+len(df_classify.index)] for i in range(0, len(all_list), len(df_classify.index))]
        df_classify1 = pd.DataFrame(chunk)
        dfclassify=df_classify1.transpose()
        dfclassify['ObjType'] = df_classify['ObjType']
        dfclassify['rangeIdx'] = df_merged['rangeIdx']
        dfclassify['dpplIdx'] = df_merged['dpplIdx']
        dfclassify['pkArray'] = df_merged['pkArray']
        dfclassify['Range'] = df_merged['Range']
        dfclassify['Coordinates.X'] = df_merged['Coordinates.X']
        dfclassify['Coordinates.Y'] = df_merged['Coordinates.Y']
        dfclassify["pkValue1.Real"] = df_merged["pkValue1.Real"]
        dfclassify["pkValue1.Imaginary"] = df_merged['pkValue1.Imaginary']
        dfclassify["pkValue2.Real"] = df_merged["pkValue2.Real"]
        dfclassify["pkValue2.Imaginary"] = df_merged['pkValue2.Imaginary']
        dfclassify["pkValue3.Real"] = df_merged["pkValue3.Real"]
        dfclassify["pkValue3.Imaginary"] = df_merged['pkValue3.Imaginary']
        dfclassify["pkValue4.Real"] = df_merged["pkValue4.Real"]
        dfclassify["pkValue4.Imaginary"] = df_merged['pkValue4.Imaginary']
        
        dfclassify['Frame'] = df_merged['Frame']
        dfclassify['Range'] = df_merged['Range']
        return dfclassify


# Load all files from json to a dataframe after 
def startdf(json_data,Label):
    df_1= pd.DataFrame()
    df1 = pd.DataFrame((json_data))
    for j in range(len(df1.index)):
        df2 = json_normalize(df1['ObjectList'][j])
        df2['ObjType'] = df2['ObjType'].map(lambda ObjType: Label)        
        #df2 = df2[df2.Range <= UIdx]
        #df2 = df2[df2.Range >= LIdx]
        #df2 = df2[df2.dpplIdx == 1]   
        df_1 = pd.concat([df2,df_1],ignore_index =True)
    df_1 = df_1.sort_values(by=['Frame'],ascending = True)
    df_1 = df_1.reset_index(drop=True)
    df_1['pkArray'] = df_1['pkArray'].map(lambda x:np.transpose(x))
    df_final = mergedframe1(df_1)
    return df_final


def labelData(df, x1, x2, y1, y2, label):
    df['ObjType'] = df['ObjType'].map(lambda x : 0)

    for index, row in df.iterrows():
        if ((row['Coordinates.X'] > x1) & (row['Coordinates.X'] < x2) 
        & (row['Coordinates.Y'] > y1) & (row['Coordinates.Y'] < y2)):
            df.at[index, 'ObjType'] = label
        else:
            df.at[index, 'ObjType'] = 0
    return df


# Shuffle two arrays in unison
def unison_shuffled_copies(array1, array2):
    assert len(array1) == len(array2)
    p = np.random.permutation(len(array1))
    return array1[p], array2[p]


def padHumanRows(df):
  for ind, row in df.iterrows():
    if ((ind % 10) == 0):
      count = 0
      while ((count < 10) and (df.iloc[ind + count]['Frame'] == int(ind/10))):
        count = count + 1
        if ((ind + count) == len(df.index)):
          break
      while (count < 10):
        row = pd.DataFrame({"0": 0, "1": 0, "2": 0, "3": 0, "4": 0, "5": 0, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "18": 0, "19": 0, "20": 0, "21": 0, "22": 0, "23": 0, "24": 0, "25": 0, "26": 0, "27": 0, "28": 0, "29": 0, "30": 0, "31": 0, "32": 0, "33": 0, "34": 0, "rangeIdx":0, "dpplIdx":0, "Frame": ind/10, "ObjType":0}, index=[ind+count-1+0.5])
        df = df.append(row, ignore_index=False)
        df = df.sort_index().reset_index(drop=True)
        df = df.reindex(["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30", "31", "32", "33", "34", "rangeIdx", "dpplIdx", "Frame", "ObjType"], axis=1)
        count = count + 1
  return df

def padCarRows(df):
  for ind, row in df.iterrows():
    if ((ind % 10) == 0):
      count = 0
      while ((count < 10) and (df.iloc[ind + count]['Frame'] == int(ind/10))):
        count = count + 1
        if ((ind + count) == len(df.index)):
          break
      while (count < 10):
        row = pd.DataFrame({0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, "rangeIdx":0, "dpplIdx":0, "Frame": ind/10, "ObjType":0}, index=[ind+count-1+0.5])
        df = df.append(row, ignore_index=False)
        df = df.sort_index().reset_index(drop=True)
        df = df.reindex([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, "rangeIdx", "dpplIdx", "Frame", "ObjType"], axis=1)
        count = count + 1
  return df


def padRows(df, obj):
  if (obj == "car"):
    while (df.shape[0] != ((df['Frame'].max() + 1)*10)):
      df = padCarRows(df)
  else:
    while (df.shape[0] != ((df['Frame'].max() + 1)*10)):
      df = padHumanRows(df)  
  return df


def truncateColumns(df):
  df = df.drop(['Range', 'pkArray', 'Coordinates.X', 'Coordinates.Y', 'pkValue1.Real', 
           'pkValue1.Imaginary', 'pkValue2.Real', 'pkValue2.Imaginary', 
           'pkValue3.Real', 'pkValue3.Imaginary', 'pkValue4.Real', 
           'pkValue4.Imaginary', 'ObjType'], axis=1)
  return df


def dfTo2DArray(df):
  df = df[df.columns[0:38]].values
  return df

#to correct the dppl idx
def correctDpplIdx(x):
    if x <=31:
        return x
    if x>31:
        return x-64


# frameTo128x64() function converts a 2D input array
# where rows == objects in json file and cols == 0,1,..34 peak array elements, rangeIdx, dopplIdx, Frame.
# The 2D array is converted to 3D array of size no. of frames * 128 * 64

def frameTo128x64(arr):
    count = 0
    new3DArray = np.empty((1, 128, 64))
    newLabelArr = np.empty((1, 128, 64))
    arr128x64 = np.zeros((128, 64))
    labelArr = np.zeros((128, 64))
    for row in range(0, np.shape(arr)[0], 1):
        if count<=10:
            rangeIndex = int(arr[row][35])
            dopplerIndex = int(correctDpplIdx(arr[row][36]))
            label = arr[row][37]
            r = 0
            
            for rangeID in range(rangeIndex-3, rangeIndex+4, 1):
                d = 0
                for dopplID in range(dopplerIndex-2, dopplerIndex+3, 1):
                    if ((0 <= rangeID <= 127) and (-32 <= dopplID <= 31)):
                        if(arr128x64[rangeID][dopplID+32] < arr[row][(d * 7) + r]):
                            arr128x64[rangeID][dopplID+32] = arr[row][(d * 7) + r]
                        if(labelArr[rangeID][dopplID+32] == 0):
                            labelArr[rangeID][dopplID+32] = label
                    d = d + 1
                r = r + 1
        count= count + 1
        if count == 10:
            count = 0
            arr128x64 = np.reshape(arr128x64, (1, 128, 64))
            labelArr = np.reshape(arr128x64, (1, 128, 64))
            new3DArray = np.append(new3DArray, arr128x64, axis=0)
            newLabelArr = np.append(newLabelArr, labelArr, axis=0)
            arr128x64 = np.zeros((128, 64))
            labelArr = np.zeros((128, 64))
    return new3DArray[1:, :, :], newLabelArr[1:, :, :]

Load Human data from json files to pandas dataframe

In [ ]:
humanDay1_2pm = loadData("../../../../media/viren/RadarDataset/human/human_2pm.json")
humanDay1_5pm = loadData("../../../../media/viren/RadarDataset/human/human_5pm.json")
humanDay1_9pm = loadData("../../../../media/viren/RadarDataset/human/human_9pm.json")
humanDay1_3pm = loadData("../../../../media/viren/RadarDataset/human/human_3pm.json")
humanDay1_6pm = loadData("../../../../media/viren/RadarDataset/human/human_6pm.json")

Load CarV data from json files to pandas dataframe

In [ ]:
carVday2_5 = startdf(openfile("../../../../media/viren/RadarDataset/carV/Car Vertical_5_28_02_2020 12_38_17.json"), 2)
carVday1_6 = startdf(openfile("../../../../media/viren/RadarDataset/carV/Car Vertical_6_27_01_2020 12_42_51.json"), 2)
carVday1_10 = startdf(openfile("../../../../media/viren/RadarDataset/carV/Car Vertical_10_27_01_2020 12_46_59.json"), 2)
carVday2_11 = startdf(openfile("../../../../media/viren/RadarDataset/carV/Car Vertical_11_28_02_2020 13_00_06.json"), 2)
carVday2_15 = startdf(openfile("../../../../media/viren/RadarDataset/carV/Car Vertical_15_28_02_2020 13_23_56.json"), 2)
carVday1_16 = startdf(openfile("../../../../media/viren/RadarDataset/carV/Car Vertical_16_27_01_2020 12_53_52.json"), 2)
carVday2_20 = startdf(openfile("../../../../media/viren/RadarDataset/carV/Car Vertical_20_28_02_2020 15_38_52.json"), 2)
carVday1_19 = startdf(openfile("../../../../media/viren/RadarDataset/carV/Car Vertical_19_27_01_2020 12_57_35.json"), 2)

Load CarH data fron json files to pandas dataframe

In [ ]:
carHday1_5 = startdf(openfile("../../../../media/viren/RadarDataset/carH/Car Horizontal_5_27_01_2020 13_05_27.json"), 3)
carHday1_7 = startdf(openfile("../../../../media/viren/RadarDataset/carH/Car Horizontal_7_27_01_2020 13_07_36.json"), 3)
carHday1_11 = startdf(openfile("../../../../media/viren/RadarDataset/carH/Car Horizontal_11_27_01_2020 13_40_38.json"), 3)
carHday1_17 = startdf(openfile("../../../../media/viren/RadarDataset/carH/Car Horizontal_17_27_01_2020 13_17_50.json"), 3)
carHday1_20 = startdf(openfile("../../../../media/viren/RadarDataset/carH/Car Horizontal_20_27_01_2020 13_21_11.json"), 3)
carHday2_2 = startdf(openfile("../../../../media/viren/RadarDataset/carH/Car Horizontal_2_12_03_2020 14_50_24.json"), 3)
carHday2_4 = startdf(openfile("../../../../media/viren/RadarDataset/carH/Car Horizontal_4_12_03_2020 14_53_38.json"), 3)
carHday2_6 = startdf(openfile("../../../../media/viren/RadarDataset/carH/Car Horizontal_6_12_03_2020 15_00_25.json"), 3)

Convert pandas dataframe to 3D array of input samples and labels. Each label array is 128x64 array of pixel wise labels.

In [ ]:
arrHumanDay1_2pm, labelHumanDay1_2pm = frameTo128x64(dfTo2DArray(padRows(labelData(humanDay1_2pm, -0.5, 0.5, 0.1, 7.9, 1), "human")))
arrHumanDay1_5pm, labelHumanDay1_5pm = frameTo128x64(dfTo2DArray(padRows(labelData(humanDay1_5pm, -0.5, 0.5, 0.1, 7.9, 1), "human")))
arrHumanDay1_9pm, labelHumanDay1_9pm = frameTo128x64(dfTo2DArray(padRows(labelData(humanDay1_9pm, -0.5, 0.5, 0.1, 7.9, 1), "human")))
arrHumanDay1_3pm, labelHumanDay1_3pm = frameTo128x64(dfTo2DArray(padRows(labelData(humanDay1_3pm, -0.5, 0.5, 0.1, 7.9, 1), "human")))
arrHumanDay1_6pm, labelHumanDay1_6pm = frameTo128x64(dfTo2DArray(padRows(labelData(humanDay1_6pm, -0.5, 0.5, 0.1, 7.9, 1), "human")))

In [ ]:
arrCarVday1_6, labelCarVday1_6 = frameTo128x64(dfTo2DArray(padRows(labelData(carVday1_6, -1.5, 1.5, 5.5, 7.5, 2), "car")))
arrCarVday1_10, labelCarVday1_10 = frameTo128x64(dfTo2DArray(padRows(labelData(carVday1_10, -1.5, 1.5, 9.5, 11.5, 2), "car")))
arrCarVday1_16, labelCarVday1_16 = frameTo128x64(dfTo2DArray(padRows(labelData(carVday1_16, -1.5, 1.5, 15.5, 17.5, 2), "car")))
arrCarVday1_19, labelCarVday1_19 = frameTo128x64(dfTo2DArray(padRows(labelData(carVday1_19, -1.5, 1.5, 18.5, 20.5, 2), "car")))
arrCarVday2_5, labelCarVday2_5 = frameTo128x64(dfTo2DArray(padRows(labelData(carVday2_5, -1.5, 1.5, 4.5, 6.5, 2), "car")))
arrCarVday2_15, labelCarVday2_15 = frameTo128x64(dfTo2DArray(padRows(labelData(carVday2_15, -1.5, 1.5, 14.5, 16.5, 2), "car")))
arrCarVday2_11, labelCarVday2_11 = frameTo128x64(dfTo2DArray(padRows(labelData(carVday2_11, -1.5, 1.5, 10.5, 12.5, 2), "car")))
arrCarVday2_20, labelCarVday2_20 = frameTo128x64(dfTo2DArray(padRows(labelData(carVday2_20, -1.5, 1.5, 19.5, 21.5, 2), "car")))

In [ ]:
arrCarHday1_5, labelCarHday1_5 = frameTo128x64(dfTo2DArray(padRows(labelData(carHday1_5, -3, 3, 4.5, 7, 3), "car")))
arrCarHday1_7, labelCarHday1_7 = frameTo128x64(dfTo2DArray(padRows(labelData(carHday1_7, -3, 3, 6.5, 9, 3), "car")))
arrCarHday1_11, labelCarHday1_11 = frameTo128x64(dfTo2DArray(padRows(labelData(carHday1_11, -3, 3, 10.5, 13, 3), "car")))
arrCarHday1_17, labelCarHday1_17 = frameTo128x64(dfTo2DArray(padRows(labelData(carHday1_17, -3, 3, 16.5, 19, 3), "car")))
arrCarHday1_20, labelCarHday1_20 = frameTo128x64(dfTo2DArray(padRows(labelData(carHday1_20, -3, 3, 19.5, 22, 3), "car")))
arrCarHday2_2, labelCarHday2_2 = frameTo128x64(dfTo2DArray(padRows(labelData(carHday2_2, -3, 3, 1.5, 4, 3), "car")))
arrCarHday2_4, labelCarHday2_4 = frameTo128x64(dfTo2DArray(padRows(labelData(carHday2_4, -3, 3, 3.5, 6, 3), "car")))
arrCarHday2_6, labelCarHday2_6 = frameTo128x64(dfTo2DArray(padRows(labelData(carHday2_6, -3, 3, 5.5, 8, 3), "car")))

In [ ]:
print("Human Data:\narrHumanDay1_2pm:", arrHumanDay1_2pm.shape, "  labelHumanDay1_2pm:", labelHumanDay1_2pm.shape, 
      "\narrHumanDay1_5pm:", arrHumanDay1_5pm.shape, "  labelHumanDay1_5pm:", labelHumanDay1_5pm.shape, "\narrHumanDay1_9pm:", 
      arrHumanDay1_9pm.shape, "  labelHumanDay1_9pm:", labelHumanDay1_9pm.shape, "\narrHumanDay1_3pm:", arrHumanDay1_3pm.shape,
      "  labelHumanDay1_3pm:", labelHumanDay1_3pm.shape, "\narrHumanDay1_6pm:", arrHumanDay1_6pm.shape, "  labelHumanDay1_6pm:", labelHumanDay1_6pm.shape)

print("\ncarV Data:\narrCarVday1_6:", arrCarVday1_6.shape, "  labelCarVday1_6:", labelCarVday1_6.shape,
      "\narrCarVday1_10:", arrCarVday1_10.shape, "  labelCarVday1_10:", labelCarVday1_10.shape, "\narrCarVday1_16:", 
      arrCarVday1_16.shape, "  labelCarVday1_16:", labelCarVday1_16.shape, "\narrCarVday1_19:", arrCarVday1_19.shape, 
      "  labelCarVday1_19:", labelCarVday1_19.shape, "\narrCarVday2_5:", arrCarVday2_5.shape, "  labelCarVday2_5:", labelCarVday2_5.shape, "\narrCarVday2_15:", 
      arrCarVday2_15.shape, "  labelCarVday2_15:", labelCarVday2_15.shape, "\narrCarVday2_11:", arrCarVday2_11.shape, 
      "  labelCarVday2_11:", labelCarVday2_11.shape, "\narrCarVday2_20:", arrCarVday2_20.shape, "  labelCarVday2_20:", labelCarVday2_20.shape)

print("\ncarH Data:\narrCarHday1_5:", arrCarHday1_5.shape, "  labelCarHday1_5:", labelCarHday1_5.shape, "\narrCarHday1_7:", 
      arrCarHday1_7.shape, "  labelCarHday1_7:", labelCarHday1_7.shape, "\narrCarHday1_11:", arrCarHday1_11.shape, 
      "  labelCarHday1_11:", labelCarHday1_11.shape, "\narrCarHday1_17:", arrCarHday1_17.shape, "  labelCarHday1_17:", labelCarHday1_17.shape, "\narrCarHday1_20:", 
      arrCarHday1_20.shape, "  labelCarHday1_20:", labelCarHday1_20.shape, "\narrCarHday2_2:", arrCarHday2_2.shape, 
      "  labelCarHday2_2:", labelCarHday2_2.shape, "\narrCarHday2_6:", arrCarHday2_6.shape, "  labelCarHday2_6:", labelCarHday2_6.shape, "\narrCarHday2_4:", 
      arrCarHday2_4.shape, "  labelCarHday2_4:", labelCarHday2_4.shape)